In [83]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab

from docx import Document
from docx.shared import Pt
import traceback
from docx.oxml.shared import OxmlElement, qn

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import seaborn as sns
pandas.set_option('display.float_format', lambda x: '%.1f' % x)
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
df = pandas.read_csv("exo_one/exo_one.csv")

## 1. Format data

In [85]:
df.drop(['Κωδικός Πρότασης', 'Εταίρος ΕΦΔ', 'Επωνυμία Επιχείρησης', 'Περιγραφή ΚΑΔ Επένδυσης'], axis=1, inplace=True)
df.loc[df['ΠΕΠ'].str.contains('Μεσαίες'), 'ΠΕΠ'] = "Μεσαίες"
df.loc[df['ΠΕΠ'].str.contains('Μεγάλες'), 'ΠΕΠ'] = "Μεγάλες"
df.loc[df['ΠΕΠ'].str.contains('Μικρές'), 'ΠΕΠ'] = "Μικρές και Πολύ Μικρές"
df.rename(columns={'ΠΕΠ': 'Μέγεθος Επιχείρησης'}, inplace=True)
# set as dtype int to use in dict range
df['two_digit_sectors'] = df['ΚΑΔ Επένδυσης'].str[0:2].astype(int)
# df.dtypes
df.head(2)
# df['ΚΑΔ Επένδυσης'].sort_values().unique()

,ΚΑΔ Επένδυσης,Περιφέρεια Υλοποίησης,Μέγεθος Επιχείρησης,Συνολικός Προϋπολογισμός,Επιχορηγούμενος Προϋπολογισμός,Δημόσια Δαπάνη Ολοκλήρωσης,two_digit_sectors
0,23.70.12,ΘΕΣΣΑΛΙΑ,Μικρές και Πολύ Μικρές,50748.1,47450.0,21352.5,23
1,27.12.31,ΘΕΣΣΑΛΙΑ,Μικρές και Πολύ Μικρές,153000.0,153000.0,68850.0,27


In [86]:
sectors={}
for i in range(10, 34):  # range(a,b) gives [a,b)
    sectors[i] = 'Μεταποίηση'
for i in range(41, 44):
    sectors[i] = "Κατασκευές"
for i in range(45, 48):
    sectors[i] = "Εμπόριο"
for i in range(58, 64):
    sectors[i] = "ΤΠΕ"
for i in range(70, 75):
    sectors[i] = "Επαγ/Επιστ. Δρ."
sectors[86] = "Υγεία"
df['Κλάδος'] = df["two_digit_sectors"].map(sectors)
# df['Κλάδος'].head()

In [87]:
#! drop after mapping dictionary and before grouping
df.drop(['two_digit_sectors'], axis=1, inplace=True)

In [ ]:
sector_totals = df.groupby(['Κλάδος'], sort=True).sum()
# sector_totals

# Create SIZE DF

### 2.0 Group by size totals

In [102]:
size_totals = df.groupby(['Μέγεθος Επιχείρησης'], sort=False).sum().reset_index()
# size_totals
size_projects = df.groupby(['Μέγεθος Επιχείρησης'], sort=True).size()
size_projects = pd.DataFrame(size_projects, columns=['Έργα']).reset_index()
# size_projects
size_df = pd.merge(size_totals, size_projects)
size_df

,Μέγεθος Επιχείρησης,Συνολικός Προϋπολογισμός,Επιχορηγούμενος Προϋπολογισμός,Δημόσια Δαπάνη Ολοκλήρωσης,Έργα
0,Μικρές και Πολύ Μικρές,37534707.8,36644001.4,16732041.8,362
1,Μεσαίες,16394101.5,16075613.0,7286229.2,117
2,Μεγάλες,1343610.9,1332696.5,532853.9,7


## 2.1 Size percentages

In [98]:
size_percent_df = size_df.set_index('Μέγεθος Επιχείρησης').transform(lambda x: x*100/x.sum())
size_percent_df = size_percent_df.add_suffix(", %%")
size_percent_df = size_percent_df.applymap("{0:.2f}".format).astype(float)
## for single columns example format
# df['cost'] = df['cost'].map('${:,.2f}'.format)
size_percent_df = size_percent_df.reset_index()


,Μέγεθος Επιχείρησης,"Συνολικός Προϋπολογισμός, %","Επιχορηγούμενος Προϋπολογισμός, %","Δημόσια Δαπάνη Ολοκλήρωσης, %","Έργα, %"
0,Μικρές και Πολύ Μικρές,67.9,67.8,68.2,74.5
1,Μεσαίες,29.7,29.7,29.7,24.1
2,Μεγάλες,2.4,2.5,2.2,1.4


In [123]:
size_percent_df['Δημόσια Δαπάνη, % πρ/μού'] = size_df['Δημόσια Δαπάνη Ολοκλήρωσης']/size_df['Συνολικός Προϋπολογισμός']*100

In [124]:
size_percent_df

,Μέγεθος Επιχείρησης,"Συνολικός Προϋπολογισμός, %","Επιχορηγούμενος Προϋπολογισμός, %","Δημόσια Δαπάνη Ολοκλήρωσης, %","Έργα, %","Δημόσια Δαπάνη, % πρ/μού"
0,Μικρές και Πολύ Μικρές,67.9,67.8,68.2,74.5,44.6
1,Μεσαίες,29.7,29.7,29.7,24.1,44.4
2,Μεγάλες,2.4,2.5,2.2,1.4,39.7


## 2.2 Size means

In [125]:
size_mean_df = df.groupby(['Μέγεθος Επιχείρησης']).mean()
size_mean_df = size_mean_df.add_suffix(", μ.ο. χιλ. €")
size_mean_df = size_mean_df.transform(lambda x: x/1000).applymap("{0:.2f}".format).astype(float)
size_mean_df = size_mean_df.reset_index()
size_mean_df

,Μέγεθος Επιχείρησης,"Συνολικός Προϋπολογισμός, μ.ο. χιλ. €","Επιχορηγούμενος Προϋπολογισμός, μ.ο. χιλ. €","Δημόσια Δαπάνη Ολοκλήρωσης, μ.ο. χιλ. €"
0,Μεγάλες,191.9,190.4,76.1
1,Μεσαίες,140.1,137.4,62.3
2,Μικρές και Πολύ Μικρές,103.7,101.2,46.2


## 2.3 Merge totals, percentages, means

In [126]:
size_totals_perc_df = pandas.merge(size_df, size_percent_df)

In [127]:
for col in size_totals_perc_df.columns[[1, 2, 3]]:
    size_totals_perc_df[col] = size_totals_perc_df[col]/1000000 # Συνολικοί Προϋπολογισμοί, σε εκατ. ευρω
    size_totals_perc_df[col] = size_totals_perc_df[col].map('{:,.2f}'.format)
    size_totals_perc_df.rename(columns={''.join([col]): ''.join([col]) + ', εκ. €'}, inplace=True)  

size_totals_perc_df

,Μέγεθος Επιχείρησης,"Συνολικός Προϋπολογισμός, εκ. €","Επιχορηγούμενος Προϋπολογισμός, εκ. €","Δημόσια Δαπάνη Ολοκλήρωσης, εκ. €",Έργα,"Συνολικός Προϋπολογισμός, %","Επιχορηγούμενος Προϋπολογισμός, %","Δημόσια Δαπάνη Ολοκλήρωσης, %","Έργα, %","Δημόσια Δαπάνη, % πρ/μού"
0,Μικρές και Πολύ Μικρές,37.53,36.64,16.73,362,67.9,67.8,68.2,74.5,44.6
1,Μεσαίες,16.39,16.08,7.29,117,29.7,29.7,29.7,24.1,44.4
2,Μεγάλες,1.34,1.33,0.53,7,2.4,2.5,2.2,1.4,39.7


In [128]:
size_all_df = pandas.merge(size_totals_perc_df, size_mean_df)
# list(size_all_df.columns)
size_all_df = size_all_df[[
 'Μέγεθος Επιχείρησης',
 'Έργα',  'Έργα, %',
 'Συνολικός Προϋπολογισμός, εκ. €', 'Συνολικός Προϋπολογισμός, %', 'Συνολικός Προϋπολογισμός, μ.ο. χιλ. €',
 'Επιχορηγούμενος Προϋπολογισμός, εκ. €', 'Επιχορηγούμενος Προϋπολογισμός, %', 'Επιχορηγούμενος Προϋπολογισμός, μ.ο. χιλ. €',
 'Δημόσια Δαπάνη Ολοκλήρωσης, εκ. €', 'Δημόσια Δαπάνη Ολοκλήρωσης, %', 'Δημόσια Δαπάνη Ολοκλήρωσης, μ.ο. χιλ. €', 'Δημόσια Δαπάνη, % πρ/μού']]

In [129]:
size_all_df

,Μέγεθος Επιχείρησης,Έργα,"Έργα, %","Συνολικός Προϋπολογισμός, εκ. €","Συνολικός Προϋπολογισμός, %","Συνολικός Προϋπολογισμός, μ.ο. χιλ. €","Επιχορηγούμενος Προϋπολογισμός, εκ. €","Επιχορηγούμενος Προϋπολογισμός, %","Επιχορηγούμενος Προϋπολογισμός, μ.ο. χιλ. €","Δημόσια Δαπάνη Ολοκλήρωσης, εκ. €","Δημόσια Δαπάνη Ολοκλήρωσης, %","Δημόσια Δαπάνη Ολοκλήρωσης, μ.ο. χιλ. €","Δημόσια Δαπάνη, % πρ/μού"
0,Μικρές και Πολύ Μικρές,362,74.5,37.53,67.9,103.7,36.64,67.8,101.2,16.73,68.2,46.2,44.6
1,Μεσαίες,117,24.1,16.39,29.7,140.1,16.08,29.7,137.4,7.29,29.7,62.3,44.4
2,Μεγάλες,7,1.4,1.34,2.4,191.9,1.33,2.5,190.4,0.53,2.2,76.1,39.7


## 2.5 Export size  df to excel table

In [130]:
writer = pandas.ExcelWriter('exo_one/tables/Πίνακας_Μέγεθος_Εξωστρέφεια_Ι.xlsx', engine='xlsxwriter')
size_all_df.to_excel(writer,'Συγκεντρωτικά ανά Μέγεθος', index=False)
writer.save()